In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

In [2]:
df1 =  pd.read_csv("C:\\Users\\Administrator\\Documents\\RSBP\\fake-news-detection-apps\\dataset\\train.csv", sep=';')
df2 =  pd.read_csv("C:\\Users\\Administrator\\Documents\\RSBP\\fake-news-detection-apps\\dataset\\test.csv", sep=';')
df3 =  pd.read_csv("C:\\Users\\Administrator\\Documents\\RSBP\\fake-news-detection-apps\\dataset\\evaluation.csv", sep=';')
df = pd.concat([df1, df2,df3], ignore_index=True)
df.head()

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0


In [3]:
# device selection: MPS (for macOS with Apple Silicon), CUDA, or CPU
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using Device: {device}')

Using Device: cpu


In [4]:
class FakeNewsBinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(FakeNewsBinaryClassifier, self).__init__()
        self.network = nn.Sequential(nn.Linear(input_dim, 128),
                                     nn.ReLU(),
                                     nn.Linear(128, 64),
                                     nn.ReLU(),
                                     nn.Linear(64, 1),
                                     nn.Sigmoid())

    def forward(self, x):
        return self.network(x)

In [5]:
loaded_model = FakeNewsBinaryClassifier(input_dim=1000).to(device)
loaded_model.load_state_dict(torch.load('fake_news_classifier.pth', weights_only=True))
loaded_model.eval()

FakeNewsBinaryClassifier(
  (network): Sequential(
    (0): Linear(in_features=1000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [26]:
def predict(model, X):
    with torch.no_grad():
        output = model(X)
        true_percentage = output.item() * 100  # Confidence for True
        false_percentage = 100 - true_percentage  # Confidence for False
        
        print(f"Confidence: True = {true_percentage:.2f}%, False = {false_percentage:.2f}%")
        
        prediction = 'True' if true_percentage >= 50 else 'False'
    return prediction



In [25]:

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
vectorizer.fit_transform(df['title'] + " " + df['text']).toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07362936, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0480039 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10522938, 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(40587, 1000))

In [27]:
# new article for inference (no label)
sample_title = 'Palestinians switch off Christmas lights in Bethlehem in anti-Trump protest.'
sample_text = 'RAMALLAH, West Bank (Reuters) - Palestinians switched off Christmas lights at Jesus traditional birthplace.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 99.99%, False = 0.01%
Prediction: True


In [28]:
# new article for inference (no label)
sample_title = 'Russia cyber unit without Trump notifying Congress'
sample_text = 'Under legislation progressing in Congress, U.S. President Donald Trump would be required to notify lawmakers before establishing a joint U.S.-Russia cybersecurity unit—an idea that has faced bipartisan criticism. If enacted, this measure would add to a series of congressional actions either restricting the president’s authority on Russia-related issues or opposing his efforts to improve ties with Moscow. The requirement, included in the annual Intelligence Authorization Act, mandates the Trump administration to submit a report to Congress outlining the intelligence to be shared with Russia, any counterintelligence risks, and plans to address those risks. The Senate Intelligence Committee approved the provision by a 14-1 vote in July as part of the broader bill overseeing operations by the CIA and other intelligence agencies. While the committee passed the legislation earlier, its details were only recently disclosed due to the classified nature of the operations it governs.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 98.14%, False = 1.86%
Prediction: True


In [29]:
# new article for inference (no label)
sample_title = 'Mexico Passes Controversial Law Cementing Military’s Role in Drug War'
sample_text = 'Mexico’s Congress approved the Law of Internal Security on Friday, formalizing the military’s involvement in combating drug cartels despite opposition from human rights groups and the United Nations. Backed by the ruling Institutional Revolutionary Party and some National Action Party members, the legislation now awaits President Enrique Peña Nieto signature. The law establishes guidelines for military operations against cartels, a conflict that has resulted in over 100,000 deaths in the past decade. Proponents argue the law provides clear regulations for military actions, but critics, including Amnesty International and local human rights organizations, warn it risks increasing abuses and undermining police reform.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 90.86%, False = 9.14%
Prediction: True


In [30]:
# From dataset, paraphrased using chatgpt
# new article for inference (no label)
sample_title = 'Justice Department Investigates Arizona Election'
sample_text = 'The U.S. Department of Justice has launched a civil rights investigation into Arizona’s March 22 presidential primary, focusing on complaints of excessively long wait times at polling stations in areas with large minority populations. The inquiry targets Maricopa County, which reduced its polling sites from 200 in 2012 to just 60 this year, citing cost-cutting measures. The cuts led to voters waiting up to five hours to cast their ballots.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 89.78%, False = 10.22%
Prediction: True


In [31]:
# test from https://apnews.com/article/fb2e92a47f054019a2589ace78d20836
# new article for inference (no label)
sample_title = 'Trump claims astounding victory as America’s 45th president'
sample_text = 'Donald Trump secured his place as America’s 45th president on Wednesday, achieving a shocking victory over Hillary Clinton. The political newcomer and businessman capitalized on economic anxieties, racial tensions, and a deeply divided electorate to clinch the White House, despite a campaign shadowed by sexual assault allegations.Trump’s triumph ends eight years of Democratic leadership and signals potential reversals of key Obama-era policies, including the Affordable Care Act, the Iran nuclear deal, and major trade agreements. Calling for unity in his victory speech, Trump urged Americans to “come together as one united people.” Clinton conceded the race in a phone call but postponed her public remarks. Despite his earlier rallying cries of “lock her up,” Trump acknowledged her public service contributions.Breaking through the Democrats’ strongholds, Trump flipped Pennsylvania and Wisconsin, states that hadn’t backed a Republican presidential candidate since the 1980s. He also won key battlegrounds like Florida, Ohio, and North Carolina. Global stock markets and U.S. futures plunged amid uncertainty about the economic and trade policies of a Trump administration.'
# The sample was paraphrased with chatgpt

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 84.08%, False = 15.92%
Prediction: True


In [32]:
# From https://faculty.lsu.edu/fakenews/elections/sixteen.php
# no parapharasing
# new article for inference (no label)
sample_title = 'How Fake News Affected the 2016 Presidential Election'
sample_text = 'Facebook CEO Mark Zuckerberg’s appearance before the Senate’s Commerce and Judiciary committees was a critical test for the future of social media platforms that skeptical lawmakers say need tougher regulations to prevent the proliferation of fake-news and interference in domestic matters by foreign adversaries. Lawmakers grilled the 33-year-old executive on the revelation that Cambridge Analytica, a political consulting firm linked to the Trump campaign, harvested data from 87 million Facebook users to profile voters and sway the 2016 election. The hearing left many lawmakers unsatisfied and did not result in government oversight of social media platforms. But Zuckerberg said Facebook would be “investigating many apps, tens of thousands of app, and if we find suspicious activity, we’re going to conduct a full audit of those apps to understand how their using their data and if they’re doing anything improper..we’ll ban them from Facebook and we will tell everyone affected.”Zuckerberg admitted the company’s failure to stop Russian bots was one of his greatest regrets. He said Facebook tracked known Russian hacking groups in real time but that it took much longer to nail inflammatory posts by the IRA. Facebook later deployed artificial intelligence tools to detect malicious activity in elections in France, Italy, and a Senate race in Alabama. In closing, Zuckerberg said “There are people in Russia whose job is to exploit our systems. This is an arms race.” But the Cambridge Analytica debacle underscored how Facebook can be used for nefarious reasons, and how the tools social media platforms create can be exploited to manipulate users’ information. In May 2017, Facebook announced plans to hire 3,000 content reviewers.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 92.48%, False = 7.52%
Prediction: True


In [33]:
# https://www.cbsnews.com/newyork/news/ny-gun-control-measure-heads-to-assembly-for-vote/
# new article for inference (no label)
sample_title = 'New York News Gov. Cuomo Signs Nations Toughest Gun Control Legislation Into Law'
sample_text = 'Not all in the Assembly agreed that the bill was thoroughly thought through. For an issue of this importance, and its impact on our Second Amendment rights guaranteed by the United States Constitution, the legislation that was passed today lacked sufficient public input and discussion to ensure that we provide meaningful protection for our children, families and communities, Assemblyman Brian Kolb said. Stakeholders from across the state – including law enforcement, school safety officials, firearms dealers, metal health professionals, business owners, criminal profilers, law abiding gun owners and the public at large - never had the opportunity to comment or make their voices heard regarding this critical issue.'

# combine and vectorize, then move to device
sample_combined = sample_title + ' ' + sample_text
sample_vector = vectorizer.transform([sample_combined]).toarray()
sample_tensor = torch.tensor(sample_vector, dtype=torch.float32).to(device)

# run inference
print("Prediction:", predict(loaded_model, sample_tensor))

Confidence: True = 73.33%, False = 26.67%
Prediction: True
